# Get Touristflow and Scenic spots info
Requires monthly tourist flow data in the statistics database of the "[Tourism Statistics Database of the Taiwan Tourism Bureau](https://stat.taiwan.net.tw/scenicSpot)", and placed in the following path:

`data/raw/tourist_flow(month)`

In [1]:
import os
import re
import pandas as pd 

In [88]:
# input
touristflow_path = 'data/raw/tourist_flow(month)'
touristflow_path_ls = [os.path.join(touristflow_path, i) for i in os.listdir(touristflow_path) if i.find('.csv')!=-1]

# output
proccess_output_path = 'data/proccess'
final_output_path = 'data/final'

os.makedirs(proccess_output_path, exist_ok=True); os.makedirs(final_output_path, exist_ok=True)

In [121]:
# read_data & combine dataframe
spot_info_dt = {}

for idx, path in enumerate(touristflow_path_ls):
    part_df = pd.read_csv(path, encoding='utf-8-sig')
    part_df.drop(0, inplace=True) # 刪除英文名
    part_df.dropna(inplace=True) # 刪除總計行
    part_df.set_index(['年度', '月份'], inplace=True)
    if idx == 0:
        df = part_df.copy()
    else:
        df = df.join(part_df)
    
    search_county = re.search('_(\w+).csv', path)
    spot_info_dt.update({i:{'county':(search_county[1] if search_county != None else '')} for i in part_df.columns})

In [122]:
# sort & output data
df.reset_index(inplace=True)
for col in df.columns[1:]:
    df[col] = df[col].astype(int)
df.sort_values(['年度', '月份'], inplace=True)
df.to_csv(os.path.join(final_output_path, '0_touristflow.csv'), index=False)

In [123]:
df

,年度,月份,秀姑巒溪遊客中心,石梯坪,花蓮管理站遊客中心,遠雄海洋公園,鯉魚潭風景特定區,池南國家森林遊樂區,新光兆豐休閒農場,富源國家森林遊樂區,...,初鹿牧場,池上大坡池地區,國立臺灣史前文化博物館,台東海洋夢想館,卑南遺址公園,知本國家森林遊樂區,蘭嶼,水往上流遊憩區,金針山休閒農業區,台東森林公園
0,100(2011),1,17624,34829,1682,18237,55312,1306,10880,2891,...,26805,0,10330,926,11469,9993,2395,0,10626,0
1,100(2011),2,25644,38799,3231,50094,106240,3914,22334,8135,...,51535,0,15698,3447,19531,14553,2861,0,10000,0
2,100(2011),3,23097,59679,1789,17788,68109,2107,9573,4119,...,14492,0,7782,543,11122,4922,2450,0,3900,0
3,100(2011),4,27719,77848,2815,31459,93127,1916,19426,6103,...,28195,0,13223,2250,18942,7978,8509,0,3200,0
4,100(2011),5,20194,58656,2211,26573,84117,2475,12687,4137,...,17835,0,9334,1401,16331,5218,8422,0,4500,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,110(2021),1,11831,13340,9310,35685,93932,2117,13484,7202,...,33918,53336,0,83,31197,17850,1192,41405,23000,29951
121,110(2021),2,13457,20755,8989,58827,130999,2902,21629,5203,...,59272,31764,0,127,32621,22338,3937,57139,65000,40156
122,110(2021),3,12905,13509,6188,28888,83858,2166,8613,3725,...,26041,26113,0,81,31025,13790,6861,15639,24000,21176
123,110(2021),4,14545,14326,8600,47702,83154,4795,15450,3541,...,29730,43216,0,287,32947,15804,23255,29013,22000,48251


In [124]:
# spot basic info
mid_dt = df[df.columns[2:]].replace(0, pd.NA).median(skipna=True).to_dict()
mean_dt = df[df.columns[2:]].replace(0, pd.NA).mean(skipna=True).to_dict()
std_dt = df[df.columns[2:]].replace(0, pd.NA).std(skipna=True).to_dict()
max_dt = df[df.columns[2:]].max()
for col in df.columns[2:]:
    spot_info_dt[col]['num_of_zero'] = df[col].to_list().count(0)
    spot_info_dt[col]['num_of_not_zero'] = df.shape[0] - spot_info_dt[col]['num_of_zero']
    spot_info_dt[col]['median'] = round(mid_dt[col])
    spot_info_dt[col]['mean'] = round(mean_dt[col])
    spot_info_dt[col]['max'] = max_dt[col]
    spot_info_dt[col]['standard_deviation'] = round(std_dt[col])

In [137]:
# start year(month) & num of suspend business 
for col in df.columns[2:]:
    start = False
    suspend_count = 0
    for year, month, value in df[['年度', '月份', col]].values:
        if (~start & (value != 0)):
            spot_info_dt[col]['start_year'] = year
            spot_info_dt[col]['start_month'] = month
            start = True
        if (start & (value == 0)):
            suspend_count +=1

    spot_info_dt[col]['suspend_business_count'] = suspend_count

In [147]:
spot_info = pd.DataFrame(spot_info_dt).T
spot_info.index.name = 'name'
spot_info.reset_index(inplace=True)

In [150]:
spot_info[
          ['name', 'county', 'start_year', 'start_month', 'num_of_not_zero', 'num_of_zero', 
          'suspend_business_count', 'median', 'mean', 'max', 'standard_deviation']
        ].to_csv(os.path.join(final_output_path, '0_spot_info.csv'), index=False)

In [153]:
spot_info.head()

,name,county,num_of_zero,num_of_not_zero,median,mean,max,standard_deviation,start_year,start_month,suspend_business_count
0,秀姑巒溪遊客中心,花蓮縣,0,125,19396,21601,46473,8476,100(2011),1,0
1,石梯坪,花蓮縣,0,125,38799,71493,392116,78132,100(2011),1,0
2,花蓮管理站遊客中心,花蓮縣,0,125,4486,4800,9361,1411,100(2011),1,0
3,遠雄海洋公園,花蓮縣,0,125,38110,45198,185382,25401,100(2011),1,0
4,鯉魚潭風景特定區,花蓮縣,0,125,96432,104345,394542,42319,100(2011),1,0
